In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

In [2]:
df = spark.read.csv("/FileStore/tables/bank.csv",header=True,inferSchema=True)
df.printSchema()

root
-- age: integer (nullable = true)
-- job: string (nullable = true)
-- marital: string (nullable = true)
-- education: string (nullable = true)
-- default: string (nullable = true)
-- balance: integer (nullable = true)
-- housing: string (nullable = true)
-- loan: string (nullable = true)
-- contact: string (nullable = true)
-- day: integer (nullable = true)
-- month: string (nullable = true)
-- duration: integer (nullable = true)
-- campaign: integer (nullable = true)
-- pdays: integer (nullable = true)
-- previous: integer (nullable = true)
-- poutcome: string (nullable = true)
-- deposit: string (nullable = true)

In [3]:
df.head(5)
df.select('age','balance','deposit').show(5)
df.groupBy("age").count().sort("count",ascending=True).show()
df.describe('balance').show()
df.filter(df.age > 40).count()
df.groupby('marital').agg({'balance':'mean'}).show()

+---+-------+-------+
age|balance|deposit|
+---+-------+-------+
 59| 2343| yes|
 56| 45| yes|
 41| 1270| yes|
 55| 2476| yes|
 54| 184| yes|
+---+-------+-------+
only showing top 5 rows

+---+-----+
age|count|
+---+-----+
 95| 1|
 89| 1|
 92| 2|
 93| 2|
 90| 2|
 88| 2|
 87| 4|
 85| 5|
 84| 5|
 86| 5|
 83| 7|
 18| 8|
 81| 9|
 79| 10|
 82| 10|
 19| 13|
 80| 17|
 75| 18|
 74| 18|
 78| 19|
+---+-----+
only showing top 20 rows

+-------+------------------+
summary| balance|
+-------+------------------+
 count| 11162|
 mean|1528.5385235620856|
 stddev| 3225.413325946149|
 min| -6847|
 max| 81204|
+-------+------------------+

+--------+------------------+
 marital| avg(balance)|
+--------+------------------+
divorced|1371.8352668213456|
 married|1599.9275704613447|
 single|1457.2552586696986|
+--------+------------------+

In [4]:
df.filter(df.age > 40).count()

Out[8]: 4967

In [5]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
age,59,56,41,55,54
job,admin.,admin.,technician,services,admin.
marital,married,married,married,married,married
education,secondary,secondary,secondary,secondary,tertiary
default,no,no,no,no,no
balance,2343,45,1270,2476,184
housing,yes,no,yes,yes,no
loan,no,no,no,no,no
contact,unknown,unknown,unknown,unknown,unknown
day,5,5,5,5,5


In [6]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int']
df.select(numeric_features).describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
age,11162,41.231947679627304,11.913369192215518,18,95
balance,11162,1528.5385235620856,3225.413325946149,-6847,81204
day,11162,15.658036194230425,8.420739541006462,1,31
duration,11162,371.99381831213043,347.12838571630687,2,3881
campaign,11162,2.508421429851281,2.7220771816614824,1,63
pdays,11162,51.33040673714388,108.75828197197717,-1,854
previous,11162,0.8325568894463358,2.292007218670508,0,58


In [7]:
categoricalColumns = ['job','marital','education','default','housing','loan','contact','poutcome']
stages = []
for categoricalCol in categoricalColumns:
  stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
  encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()],outputCols=[categoricalCol + "classVec"])
  stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'deposit', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['age','balance','duration','campaign','pdays','previous']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [8]:
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
df.printSchema()

root
-- age: integer (nullable = true)
-- job: string (nullable = true)
-- marital: string (nullable = true)
-- education: string (nullable = true)
-- default: string (nullable = true)
-- balance: integer (nullable = true)
-- housing: string (nullable = true)
-- loan: string (nullable = true)
-- contact: string (nullable = true)
-- day: integer (nullable = true)
-- month: string (nullable = true)
-- duration: integer (nullable = true)
-- campaign: integer (nullable = true)
-- pdays: integer (nullable = true)
-- previous: integer (nullable = true)
-- poutcome: string (nullable = true)
-- deposit: string (nullable = true)
-- jobIndex: double (nullable = false)
-- jobclassVec: vector (nullable = true)
-- maritalIndex: double (nullable = false)
-- maritalclassVec: vector (nullable = true)
-- educationIndex: double (nullable = false)
-- educationclassVec: vector (nullable = true)
-- defaultIndex: double (nullable = false)
-- defaultclassVec: vector (nullable = true)
-- housingIndex: double (nullable = false)
-- housingclassVec: vector (nullable = true)
-- loanIndex: double (nullable = false)
-- loanclassVec: vector (nullable = true)
-- contactIndex: double (nullable = false)
-- contactclassVec: vector (nullable = true)
-- poutcomeIndex: double (nullable = false)
-- poutcomeclassVec: vector (nullable = true)
-- label: double (nullable = false)
-- features: vector (nullable = true)

In [9]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
age,59,56,41,55,54
job,admin.,admin.,technician,services,admin.
marital,married,married,married,married,married
education,secondary,secondary,secondary,secondary,tertiary
default,no,no,no,no,no
balance,2343,45,1270,2476,184
housing,yes,no,yes,yes,no
loan,no,no,no,no,no
contact,unknown,unknown,unknown,unknown,unknown
day,5,5,5,5,5


In [10]:
train, test = df.randomSplit([0.8,0.2],seed = 99999)
from pyspark.ml.clustering import KMeans
import numpy as np
cost = np.zeros(10)
for k in range(2,10):
  kmeans = KMeans().setK(k).setSeed(1)
  model = kmeans.fit(train)
  cost[k] = model.computeCost(train)


In [11]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import seaborn as sbs
from matplotlib.ticker import MaxNLocator

fig, ax = plt.subplots(1,1,figsize = (8,6))
ax.plot(range(1,9),cost[2:10])
ax.set_xlabel('k')
ax.set_ylabel('cost')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
display(fig)

In [12]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
kmeans = KMeans().setK(3).setSeed(999)
model = kmeans.fit(train)


In [13]:
evaluator = ClusteringEvaluator()
predictions = model.transform(train)
silhouette = evaluator.evaluate(predictions)
print("Training Dataset Performance = " + str(silhouette))


Training Dataset Performance = 0.879399553497968

In [14]:
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
  print(center)

Cluster Centers: 
[2.19323794e-01 1.79181968e-01 1.62213499e-01 1.22704825e-01
 8.69950614e-02 6.54678992e-02 3.67228061e-02 3.27972648e-02
 3.35570470e-02 2.91249842e-02 2.51994428e-02 5.62112195e-01
 3.19235153e-01 5.03482335e-01 3.18222110e-01 1.32328732e-01
 9.83031531e-01 5.16272002e-01 8.60073446e-01 7.21666456e-01
 2.13372167e-01 7.46739268e-01 1.12447765e-01 9.40863619e-02
 4.08317082e+01 7.49725972e+02 3.71516145e+02 2.51006711e+00
 5.22369254e+01 8.07648474e-01]
[2.75574113e-01 1.31524008e-01 1.79540710e-01 9.60334029e-02
 6.36743215e-02 1.02296451e-01 3.86221294e-02 3.02713987e-02
 2.40083507e-02 2.40083507e-02 2.81837161e-02 5.93945720e-01
 2.96450939e-01 4.00835073e-01 4.16492693e-01 1.37787056e-01
 9.98956159e-01 5.85594990e-01 9.37369520e-01 7.17118998e-01
 1.87891441e-01 7.16075157e-01 1.11691023e-01 1.22129436e-01
 4.40490605e+01 6.46498330e+03 4.03259916e+02 2.42588727e+00
 5.20542797e+01 9.90605428e-01]
[3.55932203e-01 5.08474576e-02 1.52542373e-01 6.77966102e-02
 3.38983051e-02 1.86440678e-01 6.77966102e-02 1.69491525e-02
 0.00000000e+00 1.69491525e-02 1.69491525e-02 6.27118644e-01
 3.38983051e-01 3.55932203e-01 5.25423729e-01 5.08474576e-02
 1.00000000e+00 6.94915254e-01 9.66101695e-01 7.45762712e-01
 1.52542373e-01 7.62711864e-01 5.08474576e-02 1.01694915e-01
 4.63050847e+01 2.67298475e+04 4.10000000e+02 2.86440678e+00
 4.03898305e+01 7.28813559e-01]

In [15]:
kmeans = KMeans().setK(3).setSeed(999)
model = kmeans.fit(test)

In [16]:
evaluator = ClusteringEvaluator()
predictions = model.transform(test)
silhouette = evaluator.evaluate(predictions)
print("Testing Dataset Performance =" + str(silhouette))

Testing Dataset Performance =0.8752390670209059